In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 10
expName = "PSI_Site_DLNN_RNAfold"
outPath = "Results"
foldName = "folds.pickle"

# modelNames = ["DLNN_3", "DLNN_5"]

epochs = 100
batch_size = 16
shuffle = False
seed = None


input_data_folder = "Data\\Aziz"

In [2]:
import os 
from Bio import SeqIO
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_dna(sequence):
    seq_encoded = np.zeros((len(sequence),4))
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "T": 3
    }
    i = 0
    for single_character in sequence:
        if(single_character.upper() in dict_nuc.keys()):
            seq_encoded[i][dict_nuc[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in DNA sequence: '+sequence)
    return seq_encoded

def one_hot_encode_rna(sequence):
    seq_encoded = np.zeros((len(sequence),4))
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "U": 3
    }
    i = 0
    for single_character in sequence:
        if(single_character.upper() in dict_nuc.keys()):
            seq_encoded[i][dict_nuc[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in RNA sequence: '+sequence)
    return seq_encoded

def one_hot_encode_rnafold(sequence):
    seq_encoded = np.zeros((len(sequence),3))
    dict_fold = {
        "(": 0,
        ")": 1,
        ".": 2
    }
    i = 0
    for single_character in sequence:
        if(single_character in dict_fold.keys()):
            seq_encoded[i][dict_fold[single_character]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in RNAfold: '+sequence)
    return seq_encoded

In [5]:
##################################################################################
##### define evaluator functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

def build_kfold_multifeature(features_1, features_2, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features_1, labels):
        X1_train, X1_test = features_1[train_index], features_1[test_index]
        X2_train, X2_test = features_2[train_index], features_2[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X1_train": X1_train,
            "X1_test": X1_test,
            "X2_train": X2_train,
            "X2_test": X2_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

def pred2label(y_pred):
    y_pred = np.round(np.clip(y_pred, 0, 1))
    return y_pred

In [6]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_RNAFold_custom_1(input_shape_1 = (21,4), input_shape_2 = (21,3),
                          conv_filters_per_layer = 5, max_kernel_length = 9, conv_strides = 1, ## 1st Convolutional layer parameters
                          max_pool_width = 2, max_pool_stride = 2, ## 1st Maxpool layer parameters
                          rnn_decode_units = 25, ## LSTM layer parameters
                          dense_decode_units = 256, ## Dense layer parameters
                          prob = 0.5, learn_rate = 0.001, loss = 'binary_crossentropy', 
                          metrics = None):
    
    # beta = 0.001
    
    assert max_kernel_length >= 3
    
    # max_kernel_length = int(np.ceil(input_shape_1[0]/2))
    
    input1 = tf.keras.layers.Input(shape=input_shape_1)
    input2 = tf.keras.layers.Input(shape=input_shape_2)
    
    ##################################################
    ##### SEQUENCE
    ##################################################
    
    #########################
    ##### LSTM
    #########################
    
    sx1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_decode_units, 
                                                             return_sequences=True, 
                                                             return_state=False))(input1)
    # sx1 = tf.keras.layers.Dropout(prob)(sx1)
    sx1 = tf.keras.layers.Flatten()(sx1)
    
    #########################
    ##### Multipath Conv
    #########################
    
    sx2 = []
    for kernel_length in range(3, max_kernel_length):
        
        sxx = tf.keras.layers.Conv1D(conv_filters_per_layer, kernel_length, strides = conv_strides)(input1)
        sxx = tf.keras.layers.Activation('relu')(sxx)
        sxx = tf.keras.layers.MaxPool1D(pool_size = max_pool_width, strides = max_pool_stride)(sxx)
        # sxx = tf.keras.layers.Dropout(prob)(sxx)
        sxx = tf.keras.layers.Flatten()(sxx)
        
        sx2.append(sxx)
    
    ##################################################
    ##### FOLD
    ##################################################
    
    fx1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_decode_units, 
                                                             return_sequences=True, 
                                                             return_state=False))(input2)
    # fx1 = tf.keras.layers.Dropout(prob)(fx1)
    fx1 = tf.keras.layers.Flatten()(fx1)
    
    #########################
    ##### Multipath Conv
    #########################
    
    fx2 = []
    for kernel_length in range(2, max_kernel_length):
        
        fxx = tf.keras.layers.Conv1D(conv_filters_per_layer, kernel_length, strides = conv_strides)(input2)
        fxx = tf.keras.layers.Activation('relu')(fxx)
        fxx = tf.keras.layers.MaxPool1D(pool_size = max_pool_width, strides = max_pool_stride)(fxx)
        # fxx = tf.keras.layers.Dropout(prob)(fxx)
        fxx = tf.keras.layers.Flatten()(fxx)
        
        fx2.append(fxx)
    
    ##################################################
    ##### Fully connected Layers
    ##################################################

    y = tf.keras.layers.Concatenate(1)([fx1]+fx2+[sx1]+sx2)
    
    y = tf.keras.layers.Dense(dense_decode_units, activation = 'relu')(y)
    y = tf.keras.layers.Dropout(prob)(y)
    
    # y = tf.keras.layers.Dense(2, activation = 'softmax')(y)
    
    y = tf.keras.layers.Dense(1, activation = 'sigmoid')(y)
    
    #########################
    ##### Generate Model from input and output
    #########################
    
    model = tf.keras.models.Model(inputs=[input1, input2], outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

    return model

In [7]:
DLNN_RNAFold_custom_1().summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 21, 3)]      0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 21, 4)]      0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 20, 5)        35          input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 19, 5)        50          input_2[0][0]                    
______________________________________________________________________________________________

In [8]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Model" : [],
    "Kernel_Length" : [],
    "Dataset" : [],
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

for root, dirs, files in os.walk(input_data_folder):
    for file in files:
        
        input_data_file = os.path.join(root, file)
        
        current_dataset_variety = input_data_file.split("\\")[-1].split(".")[0]

        csv_data = pd.read_csv(input_data_file)

        ##################################################################################
        ##### extract data from the current CSV file
        ##################################################################################

        csv_data["OHE_Sequence"] = pd.Series([one_hot_encode_rna(val) for val in csv_data["Sequence"]])
        csv_data["OHE_RNAfold"] = pd.Series([one_hot_encode_rnafold(val) for val in csv_data["RNAfold"]])

        df_positive = csv_data[csv_data['Number'].str.contains("P")]
        df_negative = csv_data[csv_data['Number'].str.contains("N")]

        positive_onehotencoded_List = (np.array(list(df_positive['OHE_Sequence'])), np.array(list(df_positive['OHE_RNAfold'])))
        negative_onehotencoded_List = (np.array(list(df_negative['OHE_Sequence'])), np.array(list(df_negative['OHE_RNAfold'])))

        print("\n======================================================================")
        print("\nFile:", input_data_file)
        print("Positive:", len(positive_onehotencoded_List[0]))
        print("Negative:", len(negative_onehotencoded_List[0]))

        ##################################################################################
        ##### Generate Folds from dataset, and store to file
        ##################################################################################

        ## create the features and labels datasets for the training
        input_size_1 = (len(positive_onehotencoded_List[0][0]), 4)
        input_size_2 = (len(positive_onehotencoded_List[0][0]), 3)

        labels = np.concatenate((np.ones((positive_onehotencoded_List[0].shape[0], 1), 
                                         dtype=np.float32), 
                                 np.zeros((negative_onehotencoded_List[0].shape[0], 1), 
                                          dtype=np.float32)), 
                                axis=0)
        features_1 = np.concatenate((positive_onehotencoded_List[0], 
                                     negative_onehotencoded_List[0]), 
                                    axis=0)
        features_2 = np.concatenate((positive_onehotencoded_List[1], 
                                     negative_onehotencoded_List[1]), 
                                    axis=0)

        folds = build_kfold_multifeature(features_1, features_2, labels, 
                                         k=n_fold, shuffle=shuffle, seed=seed)

        ## Write the k-fold dataset to file
        foldPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold))
        if(not os.path.isdir(foldPath)):
            os.makedirs(foldPath)
        pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

        ## Create and set directory to save model
        modelPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold), "models")
        if(not os.path.isdir(modelPath)):
            os.makedirs(modelPath)

        ##################################################################################
        ##### TRAIN and PREDICT for every Fold, using models
        ##################################################################################

        # fold counter
        i = 0

        for fold in folds:

            print("\nTrain/Test model "+current_dataset_variety+" on Fold #"+str(i)+".")

            kernel_length = 3
            ## Generate model using function
        #             model = Conv_LSTM_DLNN(input_shape = input_size, conv_filters_per_layer = 50, kernel_length = kernel_length, 
        #                                    lstm_decode_units = 50, max_pool_width = 2, max_pool_stride = 2, dense_decode_units = 50,
        #                                    learn_rate = 0.0001, prob = 0.5, loss='binary_crossentropy', metrics=None)

            model = DLNN_RNAFold_custom_1(input_shape_1 = input_size_1, input_shape_2 = input_size_2)

            ## Define the model callbacks for early stopping and saving the model. Then train model
        #     modelCallbacks = [
        #         tf.keras.callbacks.ModelCheckpoint(os.path.join(modelPath, "{}_bestModel-fold{}.hdf5".format(current_dataset_variety, i)),
        #                                            monitor = 'val_loss', verbose = 1, save_best_only = True, 
        #                                            save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
        #         tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, 
        #                                          mode = 'auto', baseline = None, restore_best_weights = False)
        #     ]
        #     model.fit(x = fold["X_train"], y = fold["y_train"], batch_size = batch_size, epochs = epochs, verbose = 1, 
        #               callbacks = modelCallbacks, validation_split=0.2)

            model.fit(x = [fold["X1_train"], fold["X2_train"]], y = fold["y_train"], batch_size = batch_size, epochs = epochs, verbose = 1, 
                      validation_split=0.2)

            ##################################################################################
            ##### Prediction and metrics for TRAIN dataset
            ##################################################################################

            y_pred = model.predict([fold["X1_train"], fold["X2_train"]])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_train"], label_pred)
            prec = precision_score(fold["y_train"],label_pred)

            conf = confusion_matrix(fold["y_train"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
            auc = roc_auc_score(fold["y_train"], y_pred)

            evaluations["Model"].append(current_dataset_variety)
            evaluations["Kernel_Length"].append(kernel_length)
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Train")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            ##################################################################################
            ##### Prediction and metrics for TEST dataset
            ##################################################################################

            y_pred = model.predict([fold["X1_test"], fold["X2_test"]])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_test"], label_pred)
            prec = precision_score(fold["y_test"],label_pred)

            conf = confusion_matrix(fold["y_test"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
            auc = roc_auc_score(fold["y_test"], y_pred)

            evaluations["Model"].append(current_dataset_variety)
            evaluations["Kernel_Length"].append(kernel_length)
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Test")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            i = i+1
            del model
            tf.keras.backend.clear_session()

        ##################################################################################
        ##### Dump evaluations to a file
        ##################################################################################

        evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
        if(not os.path.isdir(evalPath)):
            os.makedirs(evalPath)

        pickle.dump(evaluations,
                    open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "wb"))



File: Data\Aziz\HS_990.csv
Positive: 495
Negative: 495

Train/Test model HS_990 on Fold #0.
Epoch 1/100
45/45 [==============================] - 2s 55ms/step - loss: 0.6950 - val_loss: 0.8830
Epoch 2/100
45/45 [==============================] - 1s 19ms/step - loss: 0.6469 - val_loss: 1.0142
Epoch 3/100
45/45 [==============================] - 1s 20ms/step - loss: 0.6135 - val_loss: 0.9821
Epoch 4/100
45/45 [==============================] - 1s 20ms/step - loss: 0.6025 - val_loss: 0.9483
Epoch 5/100
45/45 [==============================] - 1s 21ms/step - loss: 0.5739 - val_loss: 0.9438
Epoch 6/100
45/45 [==============================] - 1s 20ms/step - loss: 0.5538 - val_loss: 0.9400
Epoch 7/100
45/45 [==============================] - 1s 19ms/step - loss: 0.5210 - val_loss: 0.6617
Epoch 8/100
45/45 [==============================] - 1s 18ms/step - loss: 0.5231 - val_loss: 0.9022
Epoch 9/100
45/45 [==============================] - 1s 19ms/step - loss: 0.4841 - val_loss: 1.2161
Epoch 

Epoch 61/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0221 - val_loss: 3.6887
Epoch 62/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0181 - val_loss: 4.0131
Epoch 63/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0130 - val_loss: 3.8755
Epoch 64/100
45/45 [==============================] - 1s 16ms/step - loss: 0.0188 - val_loss: 4.0592
Epoch 65/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0128 - val_loss: 3.5427
Epoch 66/100
45/45 [==============================] - 1s 16ms/step - loss: 0.0172 - val_loss: 4.5481
Epoch 67/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0128 - val_loss: 4.4574
Epoch 68/100
45/45 [==============================] - 1s 16ms/step - loss: 0.0124 - val_loss: 4.2370
Epoch 69/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0076 - val_loss: 4.2623
Epoch 70/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0123 - val_los

Epoch 42/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0171 - val_loss: 3.2955
Epoch 43/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0208 - val_loss: 2.8188
Epoch 44/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0156 - val_loss: 2.8961
Epoch 45/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0299 - val_loss: 2.8732
Epoch 46/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0216 - val_loss: 2.6894
Epoch 47/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0199 - val_loss: 3.7459
Epoch 48/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0129 - val_loss: 3.4638
Epoch 49/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0164 - val_loss: 3.3374
Epoch 50/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0122 - val_loss: 3.0931
Epoch 51/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0170 - val_los

Epoch 23/100
45/45 [==============================] - 1s 18ms/step - loss: 0.1265 - val_loss: 1.8480
Epoch 24/100
45/45 [==============================] - 1s 18ms/step - loss: 0.1120 - val_loss: 2.4887
Epoch 25/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0922 - val_loss: 2.0597
Epoch 26/100
45/45 [==============================] - 1s 16ms/step - loss: 0.0795 - val_loss: 2.5444
Epoch 27/100
45/45 [==============================] - 1s 21ms/step - loss: 0.0758 - val_loss: 2.4703
Epoch 28/100
45/45 [==============================] - 1s 19ms/step - loss: 0.0658 - val_loss: 2.5130
Epoch 29/100
45/45 [==============================] - 1s 18ms/step - loss: 0.0663 - val_loss: 2.5415
Epoch 30/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0595 - val_loss: 3.0539
Epoch 31/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0456 - val_loss: 2.7642
Epoch 32/100
45/45 [==============================] - 1s 17ms/step - loss: 0.0427 - val_los

Epoch 4/100
45/45 [==============================] - 1s 18ms/step - loss: 0.5923 - val_loss: 0.9683
Epoch 5/100
45/45 [==============================] - 1s 18ms/step - loss: 0.5734 - val_loss: 1.2357
Epoch 6/100
45/45 [==============================] - 1s 17ms/step - loss: 0.5721 - val_loss: 1.0857
Epoch 7/100
45/45 [==============================] - 1s 18ms/step - loss: 0.5271 - val_loss: 1.1405
Epoch 8/100
45/45 [==============================] - 1s 18ms/step - loss: 0.5218 - val_loss: 0.9884
Epoch 9/100
45/45 [==============================] - 1s 18ms/step - loss: 0.5075 - val_loss: 1.2368
Epoch 10/100
45/45 [==============================] - 1s 16ms/step - loss: 0.4642 - val_loss: 1.1206
Epoch 11/100
45/45 [==============================] - 1s 16ms/step - loss: 0.4406 - val_loss: 1.3012
Epoch 12/100
45/45 [==============================] - 1s 18ms/step - loss: 0.3923 - val_loss: 1.0737
Epoch 13/100
45/45 [==============================] - 1s 17ms/step - loss: 0.3797 - val_loss: 1.4

...


Train/Test model SN_628 on Fold #5.
Epoch 1/100
29/29 [==============================] - 2s 55ms/step - loss: 0.7934 - val_loss: 0.8993
Epoch 2/100
29/29 [==============================] - 1s 18ms/step - loss: 0.6363 - val_loss: 1.1564
Epoch 3/100
29/29 [==============================] - 1s 21ms/step - loss: 0.6166 - val_loss: 0.8905
Epoch 4/100
29/29 [==============================] - 1s 19ms/step - loss: 0.5615 - val_loss: 1.0670
Epoch 5/100
29/29 [==============================] - 1s 21ms/step - loss: 0.5218 - val_loss: 1.1224
Epoch 6/100
29/29 [==============================] - 1s 20ms/step - loss: 0.4877 - val_loss: 1.0875
Epoch 7/100
29/29 [==============================] - 1s 18ms/step - loss: 0.4888 - val_loss: 0.7819
Epoch 8/100
29/29 [==============================] - 1s 19ms/step - loss: 0.4384 - val_loss: 1.5261
Epoch 9/100
29/29 [==============================] - 1s 19ms/step - loss: 0.4137 - val_loss: 1.1672
Epoch 10/100
29/29 [==============================] - 1s 20ms/s

Epoch 82/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0066 - val_loss: 2.5567
Epoch 83/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0039 - val_loss: 3.9087
Epoch 84/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0030 - val_loss: 3.3588
Epoch 85/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0030 - val_loss: 3.4711
Epoch 86/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0031 - val_loss: 3.7492
Epoch 87/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0020 - val_loss: 3.1986
Epoch 88/100
29/29 [==============================] - 1s 19ms/step - loss: 8.8096e-04 - val_loss: 3.4876
Epoch 89/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0043 - val_loss: 3.1056
Epoch 90/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0038 - val_loss: 3.2830
Epoch 91/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0040 - val

Epoch 63/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0101 - val_loss: 2.9401
Epoch 64/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0064 - val_loss: 2.9251
Epoch 65/100
29/29 [==============================] - 1s 18ms/step - loss: 0.0053 - val_loss: 2.8019
Epoch 66/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0061 - val_loss: 3.7407
Epoch 67/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0057 - val_loss: 3.0141
Epoch 68/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0030 - val_loss: 3.0632
Epoch 69/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0083 - val_loss: 2.9810
Epoch 70/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0035 - val_loss: 3.1696
Epoch 71/100
29/29 [==============================] - 1s 18ms/step - loss: 0.0039 - val_loss: 3.2820
Epoch 72/100
29/29 [==============================] - 1s 18ms/step - loss: 0.0073 - val_los

Epoch 44/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0096 - val_loss: 2.1891
Epoch 45/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0084 - val_loss: 2.1034
Epoch 46/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0070 - val_loss: 2.5407
Epoch 47/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0062 - val_loss: 2.4539
Epoch 48/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0074 - val_loss: 2.6943
Epoch 49/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0072 - val_loss: 2.6389
Epoch 50/100
29/29 [==============================] - 1s 18ms/step - loss: 0.0037 - val_loss: 2.7168
Epoch 51/100
29/29 [==============================] - 1s 18ms/step - loss: 0.0059 - val_loss: 2.3733
Epoch 52/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0037 - val_loss: 2.7991
Epoch 53/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0108 - val_los

Epoch 5/100
29/29 [==============================] - 1s 19ms/step - loss: 0.5583 - val_loss: 0.7525
Epoch 6/100
29/29 [==============================] - 1s 20ms/step - loss: 0.5077 - val_loss: 0.9180
Epoch 7/100
29/29 [==============================] - 1s 19ms/step - loss: 0.4928 - val_loss: 0.8643
Epoch 8/100
29/29 [==============================] - 1s 20ms/step - loss: 0.4311 - val_loss: 1.0440
Epoch 9/100
29/29 [==============================] - 1s 21ms/step - loss: 0.4049 - val_loss: 0.9718
Epoch 10/100
29/29 [==============================] - 1s 20ms/step - loss: 0.3483 - val_loss: 1.0844
Epoch 11/100
29/29 [==============================] - 1s 21ms/step - loss: 0.3512 - val_loss: 1.2258
Epoch 12/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2928 - val_loss: 1.3618
Epoch 13/100
29/29 [==============================] - 1s 21ms/step - loss: 0.2459 - val_loss: 1.1946
Epoch 14/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2108 - val_loss: 1.

## Visualization of Evaluation

In [9]:
##################################################################################
##### Add import statement here, to make this next part of code standalone executable
##################################################################################

import os
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import numpy as np
import pandas as pd


In [10]:
##################################################################################
##### Load file and convert to dataframe for easy manipulation
##################################################################################

evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
if(not os.path.isdir(evalPath)):
    os.makedirs(evalPath)

evaluations = pickle.load(open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "rb"))

In [11]:
# evaluations["Model"] = evaluations["Model"][0:20]
# evaluations_df = pd.DataFrame.from_dict(evaluations)

In [12]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

##################################################################################
##### Group dataset (mean of metrics) by [Dataset, Model, Train_Test] combinations
##################################################################################

evaluations_df_grouped = evaluations_df.groupby(["Dataset", 
                                                 "Model", 
                                                 "Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

# DLNN_3 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_3'])]
# DLNN_5 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_5'])]

# DLNN_3_Train = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Train'])]
# DLNN_3_Test = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Test'])]

# DLNN_5_Train = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Train'])]
# DLNN_5_Test = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Test'])]

In [13]:
evaluations_df_grouped

Accuracy  Precision       AUC  Sensitivity  \
Dataset Model  Train_Test                                               
HS_990  HS_990 Test        0.542424   0.532827  0.565122     0.563250   
               Train       0.876543   0.802278  0.941823     1.000000   
MM_944  MM_944 Test        0.632408   0.607860  0.674880     0.675229   
               Train       0.905367   0.841036  0.957713     1.000000   
SN_628  SN_628 Test        0.581234   0.572457  0.638237     0.595605   
               Train       0.895616   0.827553  0.962793     1.000000   

                           Specificity       MCC  
Dataset Model  Train_Test                         
HS_990  HS_990 Test           0.532827  0.090050  
               Train          0.802278  0.777281  
MM_944  MM_944 Test           0.607860  0.273827  
               Train          0.841036  0.825740  
SN_628  SN_628 Test           0.572457  0.165305  
               Train          0.827553  0.809172

In [14]:
# ##################################################################################
# ##### Decide on metric to visualize
# ##################################################################################

# print("Metrics Available : ")
# print(list(evaluations_df_grouped.columns))

#### Select a metric to plot below:

In [15]:
# metric_to_plot = "Accuracy"

In [16]:
# ##################################################################################
# ##### Visualize with a multiple Bar chart
# ##################################################################################

# x = np.arange(len(DLNN_3_Train[metric_to_plot]))
# width = 0.15

# fig, ax = plt.subplots(figsize=(17,6))
# rects1 = ax.bar(x - (4*(width/2)), round(DLNN_3_Train[metric_to_plot]*100, 3), width, label='DLNN_3, Train')
# rects2 = ax.bar(x - (1.5*(width/2)), round(DLNN_5_Train[metric_to_plot]*100, 3), width, label='DLNN_5, Train')
# rects3 = ax.bar(x + (1.5*(width/2)), round(DLNN_3_Test[metric_to_plot]*100, 3), width, label='DLNN_3, Test')
# rects4 = ax.bar(x + (4*(width/2)), round(DLNN_5_Test[metric_to_plot]*100, 3), width, label='DLNN_5, Test')

# ## Custom y-axis tick labels
# ax.set_ylabel(metric_to_plot)
# ax.set_ylim([(math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 
#             (math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10])
# # ax.set_ylim([80, 105])

# ## Custom x-axis tick labels
# ax.set_xticks(x)
# # ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))
# # ax.set_xticklabels([m+" - "+str(n) for m,n in 
# #                         zip(DLNN_3_Train.index.get_level_values(0),DLNN_3_Train.index.get_level_values(1))],
# #                   rotation=30)
# ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))

# ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
# ax.legend(loc='upper left')

# def autolabel(rects):
#     for rect in rects:
#         height = rect.get_height()
#         ax.annotate('{}'.format(height),
#                     xy=(rect.get_x() + rect.get_width() / 2, height),
#                     xytext=(0, 3),  # 3 points vertical offset
#                     textcoords="offset points", 
#                     ha='center', va='bottom', rotation=90)

# autolabel(rects1)
# autolabel(rects2)
# autolabel(rects3)
# autolabel(rects4)

# plt.show()

### Store all metrics' plots to file

In [17]:
# ##################################################################################
# ##### Iteratively generate comparison plot using every metric
# ##################################################################################

# for metric_to_plot in list(evaluations_df_grouped.columns):
    
#     x = np.arange(len(DLNN_3_Train[metric_to_plot]))
#     width = 0.15

#     fig, ax = plt.subplots(figsize=(17,6))
#     rects1 = ax.bar(x - (4*(width/2)), round(DLNN_3_Train[metric_to_plot]*100, 3), width, label='DLNN_3, Train')
#     rects2 = ax.bar(x - (1.5*(width/2)), round(DLNN_5_Train[metric_to_plot]*100, 3), width, label='DLNN_5, Train')
#     rects3 = ax.bar(x + (1.5*(width/2)), round(DLNN_3_Test[metric_to_plot]*100, 3), width, label='DLNN_3, Test')
#     rects4 = ax.bar(x + (4*(width/2)), round(DLNN_5_Test[metric_to_plot]*100, 3), width, label='DLNN_5, Test')

#     ## Custom y-axis tick labels
#     ax.set_ylabel(metric_to_plot)
#     ax.set_ylim([(math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 
#                 (math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10])
#     # ax.set_ylim([80, 105])

#     ## Custom x-axis tick labels
#     ax.set_xticks(x)
#     # ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))
#     # ax.set_xticklabels([m+" - "+str(n) for m,n in 
#     #                         zip(DLNN_3_Train.index.get_level_values(0),DLNN_3_Train.index.get_level_values(1))],
#     #                   rotation=30)
#     ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))

#     ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
#     ax.legend(loc='upper left')

#     def autolabel(rects):
#         for rect in rects:
#             height = rect.get_height()
#             ax.annotate('{}'.format(height),
#                         xy=(rect.get_x() + rect.get_width() / 2, height),
#                         xytext=(0, 3),  # 3 points vertical offset
#                         textcoords="offset points", 
#                         ha='center', va='bottom', rotation=90)

#     autolabel(rects1)
#     autolabel(rects2)
#     autolabel(rects3)
#     autolabel(rects4)
    
#     plt.savefig(os.path.join(evalPath, "{}_DLNN_Comparison".format(metric_to_plot)))
#     plt.close()
    